# $kde fitting of histogram$

>>> ## ***load packages and launch fiji***

In [1]:
%matplotlib qt
import seaborn as sbn
import numpy as np
import matplotlib.pyplot as plt 
from scipy import stats
import h5py, tifffile
import os
import sklearn
from sklearn.neighbors import KernelDensity
from sklearn.model_selection import GridSearchCV

In [2]:
import imagej
ij = imagej.init('/home/xiao/software/Fiji.app', headless=False)
ijui = ij.ui()
ijui.showUI()
ij.py.run_macro("""run("Brightness/Contrast...")""")

Added 390 JARs to the Java classpath.


<org.scijava.script.ScriptModule at 0x7fbf2127ff50 jclass=org/scijava/script/ScriptModule jself=<LocalRef obj=0x5612d95a49b8 at 0x7fbfd42e06f0>>

>>> ## ***load image and masks***

In [156]:
#fn = '/media/xiao_usb/JuLi_2020Q1/2D_trial_reg_xanes_scan2_id_54307_2020-07-05-20-36-46.h5'
#fn = '/media/xiao_usb/JuLi_2020Q1/2D_trial_reg_xanes_scan2_id_54308_2020-07-09-11-12-16.h5'
#fn = '/media/xiao_usb/JuLi_2020Q1/2D_trial_reg_xanes_scan2_id_54311_2020-07-01-11-20-34.h5'
#fn = '/media/xiao_usb/JuLi_2020Q1/2D_trial_reg_xanes_scan2_id_54312_2020-07-03-21-00-36.h5'

#with h5py.File(fn, 'r') as f:
#    img = f['/processed_XANES2D/proc_spectrum/whiteline_pos_fit'][:]

scan_id = 54307
total_num = np.arange(1, 10)
#total_num = [1, 2, 3, 5]
fnd = f'/media/xiao_usb/JuLi_2020Q1/{scan_id}_roi_whiteline_pos_fit.tif'
img = tifffile.imread(fnd)

mask = np.zeros(img.shape)
fnm = '/media/xiao_usb/JuLi_2020Q1/{0}_roi_mask_#50_particle{1}.tif'
for particle_num in total_num:
    try:
        print(fnm.format(scan_id, particle_num))
        mask += tifffile.imread(fnm.format(scan_id, particle_num))
    except:
        pass

/media/xiao_usb/JuLi_2020Q1/54307_roi_mask_#50_particle1.tif
/media/xiao_usb/JuLi_2020Q1/54307_roi_mask_#50_particle2.tif
/media/xiao_usb/JuLi_2020Q1/54307_roi_mask_#50_particle3.tif
/media/xiao_usb/JuLi_2020Q1/54307_roi_mask_#50_particle4.tif
/media/xiao_usb/JuLi_2020Q1/54307_roi_mask_#50_particle5.tif
/media/xiao_usb/JuLi_2020Q1/54307_roi_mask_#50_particle6.tif
/media/xiao_usb/JuLi_2020Q1/54307_roi_mask_#50_particle7.tif
/media/xiao_usb/JuLi_2020Q1/54307_roi_mask_#50_particle8.tif
/media/xiao_usb/JuLi_2020Q1/54307_roi_mask_#50_particle9.tif


>>> ## ***display raw image and masks***

In [157]:
masked_img = img*mask
ijui.show(ij.py.to_java(img))
ijui.show(ij.py.to_java(masked_img))
ij.py.run_macro("""run("Enhance Contrast", "saturated=0.35")""")
ij.py.run_macro("""run("Brightness/Contrast...")""")

<org.scijava.script.ScriptModule at 0x7fbf2069bbf0 jclass=org/scijava/script/ScriptModule jself=<LocalRef obj=0x5612d90f8938 at 0x7fbf20303b30>>

>>> ## ***kde calculations***

In [85]:
print(total_num)
img1 = masked_img[(masked_img!=masked_img.min())&
                  (masked_img!=masked_img.max())]
print(masked_img.min(), masked_img.max())
print(img1.min(), img1.max())

img = masked_img[(masked_img!=masked_img.min())&
                (masked_img!=masked_img.max())].flatten()
val = np.unique(img)
print(np.abs(val[1:]-val[:-1]).min())
print(img.shape)
print(val.shape)

"""
grid = GridSearchCV(KernelDensity(atol=1e-5, rtol=1e-5, kernel="gaussian"),
                    {'bandwidth': np.logspace(-4, 0, 51)},
                    cv=5, n_jobs=20) # 20-fold cross-validation
grid.fit(img[:, None])
print(grid.best_params_)

kde1 = grid.best_estimator_
x_grid1 = np.logspace(np.log(val.min()), np.log(val.max()), 500)
pdf1 = np.exp(kde1.score_samples(x_grid1[:, None]))
#kde1.set_params(**{"bandwidth":np.abs(val[1:]-val[:-1]).min()/1.5})
"""

kde1 =  KernelDensity(bandwidth=np.abs(val[1:]-val[:-1]).min()/1.5, 
                      algorithm="kd_tree", kernel='gaussian', 
                      metric='euclidean', atol=1e-5, rtol=1e-5, 
                      breadth_first=True, leaf_size=40, metric_params=None).fit(img[:, None])
x_grid1 = np.linspace(val.min(), val.max(), 500)
pdf1 = np.exp(kde1.score_samples(x_grid1[:, None]))


"""
plt.figure(1)
sbn.distplot(img, kde=True, hist=False,
             kde_kws={"bw":np.abs(val[1:]-val[:-1]).min(), "shade":True},
             norm_hist=False, label=str(ii))
plt.figure(2)
sbn.distplot(img, kde=False, hist=True,
             kde_kws={"bw":np.abs(val[1:]-val[:-1]).min(), "shade":True},
             norm_hist=False, label=str(ii))
plt.figure(3)
sbn.distplot(img, kde=True, hist=True,
             kde_kws={"bw":np.abs(val[1:]-val[:-1]).min(), "shade":True},
             norm_hist=False, label=str(ii))
"""

[1, 2, 3, 5]
0.0 8358.0
8350.0 8356.9970703125
0.01953125
(909060,)
(265,)


'\nplt.figure(1)\nsbn.distplot(img, kde=True, hist=False,\n             kde_kws={"bw":np.abs(val[1:]-val[:-1]).min(), "shade":True},\n             norm_hist=False, label=str(ii))\nplt.figure(2)\nsbn.distplot(img, kde=False, hist=True,\n             kde_kws={"bw":np.abs(val[1:]-val[:-1]).min(), "shade":True},\n             norm_hist=False, label=str(ii))\nplt.figure(3)\nsbn.distplot(img, kde=True, hist=True,\n             kde_kws={"bw":np.abs(val[1:]-val[:-1]).min(), "shade":True},\n             norm_hist=False, label=str(ii))\n'

>>> ## ***display ked results***

In [88]:
fig, ax = plt.subplots()
ax.plot(x_grid1, pdf1, linewidth=3, alpha=0.5, label='1')
ax.hist(img, val.shape[0], fc='gray', histtype='stepfilled', alpha=0.3, density=True)
#ax.hist(img, val.shape[0], fc='gray', histtype='stepfilled', alpha=0.3)
ax.legend(loc='upper left')
ax.set_xlim(val.min()-0.001, val.max()+0.001);

>>> ## ***save kde results***

In [87]:
fnr = '/media/xiao_usb/JuLi_2020Q1/{0}_rkde_in_oi_mask_#50_particle_1&2&3&5.txt'.format(scan_id)
with open(fnr, 'w') as f:
    for x, y in zip(x_grid1, pdf1):
        f.write(str(x)+'\t'+str(y)+'\n')

In [30]:
np.abs(val[1:]-val[:-1]).min()

0.01953125

# $process peak data$

In [92]:
import glob
fnr = '/media/xiao_usb/JuLi_2020Q1/{0}_kde_in_oi_mask_#50_particle_*.txt'

In [303]:
#fn = '/media/xiao_usb/JuLi_2020Q1/2D_trial_reg_xanes_scan2_id_54307_2020-07-05-20-36-46.h5'
#fn = '/media/xiao_usb/JuLi_2020Q1/2D_trial_reg_xanes_scan2_id_54308_2020-07-09-11-12-16.h5'
#fn = '/media/xiao_usb/JuLi_2020Q1/2D_trial_reg_xanes_scan2_id_54311_2020-07-01-11-20-34.h5'
#fn = '/media/xiao_usb/JuLi_2020Q1/2D_trial_reg_xanes_scan2_id_54312_2020-07-03-21-00-36.h5'
scan_id = 54307
fns = glob.glob(fnr.format(scan_id))
print(fns)

['/media/xiao_usb/JuLi_2020Q1/54307_kde_in_oi_mask_#50_particle_1.txt', '/media/xiao_usb/JuLi_2020Q1/54307_kde_in_oi_mask_#50_particle_1&2&3.txt']


In [304]:
data = []
for fn in fns:
    data.append(np.loadtxt(fn)[3:, :])
    
idx = []
lidx = []
ridx = []
dmax = []
lval = []
rval = []
for fn in fns:
    d = np.loadtxt(fn)
    t1= np.argmax(d[:, 1])
    idx.append(t1)
    dmax.append(d[t1, 1])
    t1 = np.argmin(np.abs(d[:idx[-1], 1] - dmax[-1]/2.))
    lidx.append(t1)
    lval.append(d[t1, 1])
    t1 = np.argmin(np.abs(d[idx[-1]:, 1] - dmax[-1]/2.))
    ridx.append(t1+idx[-1])
    rval.append(d[t1+idx[-1], 1])

In [305]:
for l, r, i, d in zip(lidx, ridx, idx, data):
    print(d[i, 0])
    print(d[r, 0] - d[l, 0])
#print(d[100, 0])    

8352.273108717434
0.2989103206400614
8352.28006012024
0.3475701402803679


In [306]:
e_cen = []
for d in data:
    e_cen.append(np.sum(d[:, 1]*d[:, 0])/np.sum(d[:, 1]))
print(e_cen)

[8352.182836883483, 8352.241285062732]


In [307]:
cnt = 0
color = ['r', 'g', 'b', 'y']
plt.figure(0)
for l, r, d, fn, c in zip(lidx, ridx, data, fns, color):
    #plt.figure(cnt)
    plt.plot(d[:, 0], d[:, 1], c)
    plt.title(os.path.basename(fn).split('.')[0].split('_')[0])
    cnt += 1

"""
plt.gca().legend(['1&2&3'+'={}'.format(np.round((data[0][ridx[0], 0]-data[0][lidx[0], 0])*100)/100)+'eV',
                 '1&2&3&5'+'={}'.format(np.round((data[1][ridx[1], 0]-data[1][lidx[1], 0])*100)/100)+'eV',
                 '1'+'={}'.format(np.round((data[2][ridx[2], 0]-data[2][lidx[2], 0])*100)/100)+'eV',
                 '1&2'+'={}'.format(np.round((data[3][ridx[3], 0]-data[3][lidx[3], 0])*100)/100)+'eV'],
                loc = 'upper left')
"""

plt.gca().legend(['1'+'={}'.format(np.round((data[0][ridx[0], 0]-data[0][lidx[0], 0])*10)/10)+'eV',
                 '1&2&3'+'={}'.format(np.round((data[1][ridx[1], 0]-data[1][lidx[1], 0])*10)/10)+'eV'],
                loc = 'upper left')
ymin = plt.gca().get_ylim()[0]
ymax = plt.gca().get_ylim()[1]
plt.plot([e_cen[0], e_cen[0]], [ymin, ymax], color[0])
plt.plot([e_cen[1], e_cen[1]], [ymin, ymax], color[1])
#plt.plot([e_cen[2], e_cen[2]], [ymin, ymax], color[2])
#plt.plot([e_cen[3], e_cen[3]], [ymin, ymax], color[3])
plt.gca().set_xlabel('Energy (eV)')
plt.gca().set_ylabel('PDF')
plt.gca().tick_params(labelrotation=45)

In [308]:
cnt = 10
color = ['r', 'g', 'b', 'y']
for l, r, d, fn, c in zip(lidx, ridx, data, fns, color):
    plt.figure(cnt)
    plt.plot(d[:, 0], d[:, 1], c)
    plt.plot([e_cen[cnt-10], e_cen[cnt-10]], [ymin, ymax], color[cnt-10])
    plt.title(os.path.basename(fn).split('.')[0].split('_')[0])
    cnt += 1
    plt.gca().set_xlabel('Energy (eV)')
    plt.gca().set_ylabel('PDF')
    plt.gca().tick_params(labelrotation=45)

In [221]:
print(data[0].shape, data[1].shape)
print(data[0][0, 0], data[0][-1, 0])
print(data[1][0, 0], data[1][-1, 0])

(500, 2) (500, 2)
8351.8447265625 8354.1103515625
8350.0 8356.9970703125
